In [ ]:
import pandas as pd
from math import floor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Calcul des distances sur Terre
# !pip install geopy
from geopy.distance import geodesic as GD

# 0 - Useful functions

In [ ]:
#Convert month sales to float
def convert_month_to_float(set):
    for col in set.columns:
        if 'Month' in col:
            set[col] = set[col].str.replace(' ', '').astype(float)


#Remove useless words from column names and data
def clean_words(set):
    #Rename columns (remove 'proxy')
    set.columns = set.columns.map(lambda x: x.replace(' proxy', ''))

    #Remove column names from data
    set['Operations'] = set['Operations'].str.replace(' Operations', '')
    set['Reference'] = set['Reference'].str.replace('reference-', '')
    set['Product  Line'] = set['Product  Line'].str.replace('Product Line-', '')
    set['Division'] = set['Division'].str.replace('Division-', '')
    set['Customer Persona'] = set['Customer Persona'].str.replace('Customer Segmentation-', '')
    set['Strategic Product Family'] = set['Strategic Product Family'].str.replace('Strategic Product Family-', '')

#Create columns for first month and year
def month_year(set):
    set['First month'] = set['Date'].str[:3]
    set['Year'] = set['Date'].str[-4:]


#Replace NaNs in Month 1
def replace_nans_month_1(set):
    df_to_merge = set.loc[~set['Month 1'].isnull(), ['id_product', 'Month 1', 'Month 2', 'Month 3']]
    df_to_merge['sum'] = df_to_merge[['Month 1', 'Month 2', 'Month 3']].sum(axis=1)
    df_to_merge = df_to_merge[['id_product', 'sum']].groupby('id_product').mean()
    df_to_merge['sum'] = df_to_merge['sum'].apply(floor)
    set = set.reset_index().merge(df_to_merge, on='id_product', how='left').set_index('index')
    set.loc[set['sum'].isnull(), 'sum'] = 0
    set.loc[set['Month 1'].isnull(), 'Month 1'] = set['sum']
    set.drop(columns='sum', inplace=True)

    return set


#One hot encoding on the specified columns
def encoded_df(df, colonnes):
    df.reset_index(inplace=True)
    # Create an instance of the OneHotEncoder
    country_encoder = OneHotEncoder(handle_unknown='ignore')
    # Fit and transform the "Country" column
    encoded_country = country_encoder.fit_transform(df[colonnes])
    # Convert the encoded country column to a DataFrame
    encoded_country_df = pd.DataFrame(encoded_country.toarray(), columns=country_encoder.get_feature_names_out())
    # Concatenate the encoded country column with the original dataframe
    df_encoded = pd.concat([df, encoded_country_df], axis=1)
    # Drop the original "Country" column
    df_encoded = df_encoded.drop(colonnes, axis=1)
    df_encoded.set_index('index', inplace=True)

    return df_encoded

# Recupere la typologie du lieu de depart
def center_type(site):
    return site.split('_')[1]

# Recupere le code qui correspond au Pays
def bigramme(site):
    return site.split('_')[0]

# On transforme # On transforme les bigrammes en les Pays correspondant

def bigramme_to_pays(df):
    df['type_entrepot'] = df['Site'].apply(center_type)
    df["pays_entrepot_depart"] = df["Site"].apply(lambda x : country_dict[bigramme(x)] if bigramme(x) != 'OOS' else 'United States')
    df['pays_arrivee'] = df["Country"].apply(lambda x: country_dict[x.upper()])
    return df

# Merge 2 fois pour avoir les informations sur la logistique au depart et a l'arrivee

def merge_dep_arr(data, df_logistique):
    lpi_df_depart = df_logistique.add_suffix('_depart')
    lpi_df_arrivee = df_logistique.add_suffix('_arrivee')
    data = data.reset_index()
    df_depart = pd.merge(data, lpi_df_depart, how="inner", left_on="pays_entrepot_depart", right_on="Country_depart")
    dataset = pd.merge(df_depart, lpi_df_arrivee, how="inner", left_on="pays_arrivee", right_on="Country_arrivee").set_index('index')
    return dataset

def calculate_distance(df):
    return GD(df["lat_lon_dep"], df["lat_lon_arr"]).kilometers

# 1 - Preprocessing

In [ ]:
#Import data
# dataset = pd.read_csv('datasets_hi4/train-data.csv', sep=';', index_col='index')
# X_test = pd.read_csv('datasets_hi4/X_test.csv', sep=';', index_col='index')
# lpi_df = pd.read_csv('datasets_hi4/extra-dataset/LPIextend.csv')

dataset = pd.read_csv('train-data.csv', sep=';', index_col='index')
X_test = pd.read_csv('X_test.csv', sep=';', index_col='index')
lpi_df = pd.read_csv('extra-dataset/LPIextend.csv')

#Keep only unique rows
dataset = dataset.drop_duplicates()

#Convert month sales to float
convert_month_to_float(dataset)
convert_month_to_float(X_test)

#Remove useless words from column names and data
clean_words(dataset)
clean_words(X_test)

#Create columns for first month and year
month_year(dataset)
month_year(X_test)

In [ ]:
dataset.head()

,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference,Product Line,Division,Customer Persona,Strategic Product Family,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4,First month,Year
index,,,,,,,,,,,,,,,,,,,,
645874,156160,EUROPE,DE,NL_DC_Venray,Europe,DACH,Germany,13523,4,3,3,12,NaN,may-aug 2021,0.0,0.0,0.0,0.0,may,2021
469488,224631,CHINA,CN,CN_DC_Shanghai,China,China & HK,China,12778,4,3,14,9,NaN,jan-apr 2021,0.0,1.0,0.0,0.0,jan,2021
348904,104047,EUROPE,GB,NL_DC_HLD,Europe,UK and Ireland,United Kingdom,3513,3,2,23,7,NaN,jan-apr 2021,0.0,2.0,0.0,0.0,jan,2021
1725822,11823,EAJP,AU,AU_DC_Perth,International,Pacific,Australia,672,1,1,11,3,NaN,jan-apr 2023,0.0,0.0,0.0,0.0,jan,2023
404781,159924,EUROPE,ES,NL_DC_HLD,Europe,Iberia,Spain,3496,3,2,14,7,NaN,jan-apr 2021,0.0,0.0,0.0,0.0,jan,2021


#### Merge avec LPIextend.csv

In [ ]:
country_dict = {'NL': 'Netherlands',
    'US': 'United States',
    'AU': 'Australia',
    'CN': 'China',
    'FR': 'France',
    'IT': 'Italy',
    'SE': 'Sweden',
    'BN': 'Indonesia',
    'MN': 'Mongolia',
    'ES': 'Spain',
    'MM': 'Thailand',
    'KH': 'Cambodia',
    'LA': 'Thailand',
    'GB': 'United Kingdom',
    'SG': 'Singapore',
    'PG': 'Papua New Guinea',
    'FJ': 'Fiji',
    'PL': 'Poland',
    'HU': 'Hungary',
    'MX': 'Mexico',
    'HK': 'Hong Kong',
    'DE': 'Germany',
    'MM': 'Thailand',
    'PH': 'Philippines',
    'JP': 'Japan',
    'KR': 'Japan', # attention si on merge avec World Bank
    'TH': 'Thailand',
    'ID': 'Indonesia',
    'DK': 'Denmark',
    'VN': 'Vietnam', # attention si on merge avec World Bank
    'TW': 'China', # attention si on merge avec World Bank
    'BE': 'Belgium',
    'MY': 'Malaysia',
    'NZ': 'New Zealand',
    'AE': 'United Arab Emirates',
    'IE': 'Ireland'
}

dataset = bigramme_to_pays(dataset)
X_test = bigramme_to_pays(X_test)
dataset = merge_dep_arr(dataset, lpi_df)
X_test = merge_dep_arr(X_test, lpi_df)

In [ ]:
dataset.head()

,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference,Product Line,Division,...,Infrastructure Score_arrivee,Infrastructure Grouped Rank_arrivee,International Shipments Score_arrivee,International Shipments Grouped Rank_arrivee,Logistics Competence and Quality Score_arrivee,Logistics Competence and Quality Grouped Rank_arrivee,Timeliness Score_arrivee,Timeliness Grouped Rank_arrivee,Tracking and Tracing Score_arrivee,Tracking and Tracing Grouped Rank_arrivee
index,,,,,,,,,,,,,,,,,,,,,
645874,156160,EUROPE,DE,NL_DC_Venray,Europe,DACH,Germany,13523,4,3,...,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0
384038,139181,EUROPE,DE,NL_DC_Venray,Europe,DACH,Germany,6321,4,3,...,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0
392116,147259,EUROPE,DE,NL_DC_Venray,Europe,DACH,Germany,11181,4,3,...,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0
159613,159613,EUROPE,DE,NL_DC_HLD,Europe,DACH,Germany,10010,3,2,...,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0
2106484,147628,EUROPE,DE,NL_DC_Venray,Europe,DACH,Germany,12919,4,3,...,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0


In [ ]:
columns_a_garder = ['id_product', 'Region', 'Country', 'Site', 'Operations', 'Zone',
       'Cluster', 'Reference', 'Product  Line', 'Division', 'Customer Persona',
       'Strategic Product Family', 'Product Life cycel status', 'Date',
       'Month 1', 'Month 2', 'Month 3', 'First month', 'Year',
       'Country_depart', 'Country_arrivee', 'LPI Grouped Rank_depart', 'LPI Grouped Rank_arrivee']

columns_a_garder_dataset = columns_a_garder + ['Month 4']

In [ ]:
dataset = dataset[columns_a_garder_dataset]
X_test = X_test[columns_a_garder]

#### Calcul des distances

In [ ]:
lat_lon_dict = {
    'NL': (52.3676, 4.9041),   # Amsterdam, Netherlands
    'US': (38.8951, -77.0364),  # Washington, D.C., United States
    'AU': (-35.2809, 149.1300),  # Canberra, Australia
    'CN': (39.9042, 116.4074),  # Beijing, China
    'FR': (48.8566, 2.3522),    # Paris, France
    'IT': (41.9028, 12.4964),   # Rome, Italy
    'SE': (59.3293, 18.0686),   # Stockholm, Sweden
    'BN': (-6.2088, 106.8456),  # Jakarta, Indonesia
    'MN': (47.9212, 106.9186),  # Ulaanbaatar, Mongolia
    'ES': (40.4168, -3.7038),   # Madrid, Spain
    'MM': (13.7563, 100.5018),  # Bangkok, Thailand
    'KH': (11.5564, 104.9282),  # Phnom Penh, Cambodia
    'LA': (13.7563, 100.5018),  # Bangkok, Thailand
    'GB': (51.5099, -0.1180),   # London, United Kingdom
    'SG': (1.3521, 103.8198),   # Singapore
    'PG': (-9.4438, 147.1803),  # Port Moresby, Papua New Guinea
    'FJ': (-18.1248, 178.4501), # Suva, Fiji
    'PL': (52.2297, 21.0122),   # Warsaw, Poland
    'HU': (47.4979, 19.0402),   # Budapest, Hungary
    'MX': (19.4326, -99.1332),  # Mexico City, Mexico
    'HK': (22.3193, 114.1694),  # Hong Kong
    'DE': (52.5200, 13.4050),   # Berlin, Germany
    'PH': (13.4125, 122.5619),  # Manila, Philippines
    'JP': (35.6895, 139.6917),  # Tokyo, Japan
    'TW': (39.9042, 116.4074),  # Beijing, China
    'KR': (35.6895, 139.6917),  # Tokyo, Japan
    'TH': (13.7563, 100.5018),  # Bangkok, Thailand
    'ID': (-6.2088, 106.8456),  # Jakarta, Indonesia
    'DK': (55.6761, 12.5683),   # Copenhagen, Denmark
    'VN': (21.0285, 105.8542),  # Hanoi, Vietnam
    'BE': (50.8503, 4.3517),    # Brussels, Belgium
    'MY': (3.1390, 101.6869),   # Kuala Lumpur, Malaysia
    'NZ': (-41.2866, 174.7734), # Wellington, New Zealand
    'AE': (24.4667, 54.3667),   # Abu Dhabi, United Arab Emirates
    'IE': (53.3498, -6.2603)    # Dublin, Ireland
}

In [ ]:
dataset["lat_lon_dep"] = dataset["Site"].apply(lambda x : lat_lon_dict[bigramme(x.upper())] if bigramme(x.upper()) != 'OOS' else (38.8951, -77.0364))

dataset["lat_lon_arr"] = dataset["Country"].apply(lambda x : lat_lon_dict[x.upper()])

In [ ]:
dataset["distance"] = dataset.apply(calculate_distance, axis=1)

In [ ]:
X_test["lat_lon_dep"] = X_test["Site"].apply(lambda x : lat_lon_dict[bigramme(x.upper())] if bigramme(x.upper()) != 'OOS' else (38.8951, -77.0364))

X_test["lat_lon_arr"] = X_test["Country"].apply(lambda x : lat_lon_dict[x.upper()])

In [ ]:
X_test["distance"] = X_test.apply(calculate_distance, axis=1)

In [ ]:
#Train val
X_train, X_val, y_train, y_val = train_test_split(dataset.drop(columns=['Month 4']), dataset['Month 4'], test_size=0.2, random_state=42)

In [ ]:
X_train.head()

,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference,Product Line,Division,...,Month 3,First month,Year,Country_depart,Country_arrivee,LPI Grouped Rank_depart,LPI Grouped Rank_arrivee,lat_lon_dep,lat_lon_arr,distance
index,,,,,,,,,,,,,,,,,,,,,
1000901,21473,EAJP,AU,AU_DC_Sydney,International,Pacific,Australia,1329,1,1,...,3.0,jan,2022,Australia,Australia,19.0,19.0,"(-35.2809, 149.13)","(-35.2809, 149.13)",0.000000
293497,48640,NAM,US,US_DC_Mechanicsburg,North America,US,USA,4267,4,3,...,0.0,jan,2021,United States,United States,17.0,17.0,"(38.8951, -77.0364)","(38.8951, -77.0364)",0.000000
1259755,35470,EAJP,PH,SG_DC_ASD,International,East Asia Japan,South East Asia,2548,3,2,...,0.0,may,2022,Singapore,Philippines,1.0,43.0,"(1.3521, 103.8198)","(13.4125, 122.5619)",2458.342293
2174999,216143,EUROPE,FR,FR_DC_AUTNewlog,France,France,France,3476,3,2,...,0.0,may,2023,France,France,13.0,13.0,"(48.8566, 2.3522)","(48.8566, 2.3522)",0.000000
412809,167952,EUROPE,BE,NL_DC_Venray,Europe,BeNe,Belgium,6225,4,3,...,0.0,jan,2021,Netherlands,Belgium,3.0,7.0,"(52.3676, 4.9041)","(50.8503, 4.3517)",173.096145


In [ ]:
y_train.head()

index
1000901    0.0
293497     0.0
1259755    0.0
2174999    1.0
412809     0.0
Name: Month 4, dtype: float64

In [ ]:
#Replace Nans in column Month 1 by the mean of sales for each product
X_train = replace_nans_month_1(X_train)
X_val = replace_nans_month_1(X_val)
X_test = replace_nans_month_1(X_test)

In [ ]:
#Drop columns we don't want to keep in the model
columns_to_drop = ['id_product', 'Region', 'Operations', 'Zone', 'Cluster', 'Reference', 'Division', 'Date']
X_train = X_train.drop(columns=columns_to_drop)
X_val = X_val.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

In [ ]:
#One hot encode categorical features
categorical_features = ['Country', 'Site', 'Product  Line', 'Customer Persona',
                            'Strategic Product Family', 'Product Life cycel status', 'First month', 'Year']

X_train = encoded_df(X_train, categorical_features)
X_val = encoded_df(X_val, categorical_features)
X_test = encoded_df(X_test, categorical_features)

In [ ]:
#Add a column of 0 in val and test if they lack a column from train
for col in list(set(X_train.columns) - set(X_test.columns)):
    X_test.insert(X_train.columns.get_loc(col), col, 0.)

for col in list(set(X_train.columns) - set(X_val.columns)):
    X_val.insert(X_train.columns.get_loc(col), col, 0.)

In [ ]:
#Save data
X_train.to_csv('preprocessed_datasets/X_train.csv', sep=';')
X_test.to_csv('preprocessed_datasets/X_test.csv', sep=';')
X_val.to_csv('preprocessed_datasets/X_val.csv', sep=';')

y_train.to_csv('preprocessed_datasets/y_train.csv', sep=';')
y_val.to_csv('preprocessed_datasets/y_val.csv', sep=';')